## 1. Instalamos dependencias y librerías

⏱️ **Tiempo estimado: 2-3 minutos**

### ¿Qué vamos a hacer?
Instalamos las librerías necesarias para:
- Leer PDFs
- Crear embeddings (representaciones numéricas del texto)
- Usar el modelo Gemini de Google
- Construir un grafo de estados (agentico workflow)

In [ ]:
%pip install -U -q langchain>=0.2.5 langchain-community>=0.2.0 langchain-text-splitters>=0.2.0 langchain-google-genai>=0.0.10 chromadb>=0.5.0 tiktoken>=0.7.0 pypdf>=4 python-dotenv>=1.0.1

## 2. Importamos las librerías necesarias

⏱️ **Tiempo estimado: 1 minuto**

### Desglose de importaciones:
- **PyPDFLoader**: Lee PDFs
- **RecursiveCharacterTextSplitter**: Divide el texto en chunks (trozos)
- **Chroma**: Vector database para almacenar embeddings
- **GoogleGenerativeAIEmbeddings**: Convierte texto en vectores
- **ChatGoogleGenerativeAI**: Modelo de lenguaje Gemini
- **tqdm**: Barra de progreso

In [ ]:
import os
from pathlib import Path
from typing import List

from dotenv import load_dotenv
load_dotenv()  # Carga variables de entorno (GOOGLE_API_KEY, etc.)

# LangChain loaders, splitters, vectorstore, LLM/embeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

# Embeddings y modelo de chat con Google Gemini. Teneis las pistas por el notebook. Hasta aqui puedo leer
from langchain_google_genai import <FILL_IN>, <FILL_IN>


# Utilidad
from tqdm import tqdm

## 3. Configuramos el entorno

⏱️ **Tiempo estimado: 3-5 minutos**

### Conceptos clave:
- **CHUNK_SIZE**: Tamaño de cada fragmento de texto (ej: 1000 caracteres)
- **CHUNK_OVERLAP**: Solapamiento entre chunks (ej: 200 caracteres) para mantener contexto
- **TOP_K**: Número de documentos más relevantes a recuperar
- **Modelos**: Necesitas elegir embeddings y modelo de chat

💡 **Tip**: Valores típicos:
- CHUNK_SIZE: 500-2000
- CHUNK_OVERLAP: 100-400 (~20% del CHUNK_SIZE)
- Embedding: "models/text-embedding-004"
- Chat: "gemini-2.5-flash" o "gemini-1.5-pro"

In [ ]:
# 🛠️ Configuración
PDF_DIR = Path("./docs")             # <- Carpeta con PDFs
PERSIST_DIR = Path("./chroma_pdfs_gemini")  # Donde se guardará Chroma
PERSIST_DIR.mkdir(parents=True, exist_ok=True)

# <FILL_IN> Completa estos valores basándote en los tips
CHUNK_SIZE = <FILL_IN>  # Tamaño de cada chunk (trozo) de texto. Rango: 500-2000
CHUNK_OVERLAP = <FILL_IN>  # Solapamiento entre chunks. Recomendado: ~20% del CHUNK_SIZE
TOP_K = 4  # Número de documentos recuperados (mantén este valor)

# Modelos de Gemini
EMBEDDING_MODEL = "<FILL_IN>"  # Modelo de embeddings (usa: "models/text-embedding-004")
CHAT_MODEL = "<FILL_IN>"  # Modelo de chat (prueba: "gemini-2.5-flash")

# Verificar clave - SIN ESTO, NO FUNCIONA
assert os.getenv("GOOGLE_API_KEY"), "❌ Falta GOOGLE_API_KEY en variables de entorno o .env"
print(f"✅ GOOGLE_API_KEY detectada")
print(f"📁 Carpeta PDFs: {PDF_DIR.resolve()}")
print(f"🗂️  Persistencia Chroma: {PERSIST_DIR.resolve()}")
print(f"🔧 Configuración: CHUNK_SIZE={CHUNK_SIZE}, OVERLAP={CHUNK_OVERLAP}, TOP_K={TOP_K}")

## 4. Cargamos los PDFs

⏱️ **Tiempo estimado: 5-10 minutos** (según cantidad de PDFs)

### ¿Qué ocurre aquí?
1. Lee todos los PDFs de la carpeta `./docs`
2. Añade metadatos (origen del documento) a cada página
3. Retorna lista de documentos cargados

🎯 **Objetivo**: Tener una lista de documentos listos para procesar

💡 **Tip de debugging**: Si no ves docs, verifica que:
- Los PDFs están en `./docs`
- Los PDFs no están corruptos
- Tienes permisos de lectura

In [ ]:
def load_pdfs_from_dir(directory: Path, recursive: bool = True):
    """Carga todos los PDFs de una carpeta de forma recursiva."""
    pattern = "**/*.pdf" if recursive else "*.pdf"
    pdf_paths = sorted([p for p in directory.glob(pattern) if p.is_file()])
    all_docs = []
    for pdf in tqdm(<FILL_IN>, desc="Cargando PDFs"):
        try:
            docs = PyPDFLoader(str(pdf)).load()
            # Añadimos metadatos útiles
            for d in docs:
                d.metadata = d.metadata or {}
                d.metadata["source"] = str(pdf.resolve())
            all_docs.extend(docs)
        except Exception as e:
            print(f"⚠️ Error leyendo {pdf}: {e}")
    print(f"📚 Documentos (páginas) cargados: {len(all_docs)}")
    return all_docs

raw_docs = load_pdfs_from_dir(PDF_DIR, recursive=True)

## 5. Chunking.

⏱️ **Tiempo estimado: 1-2 minutos**

### ¿Por qué hacer chunking?
- Los modelos tienen límite de tokens (palabras)
- Dividir en trozos permite recuperar partes relevantes
- El solapamiento preserva contexto entre chunks

### Parámetros de recursión:
- `separators=["\n\n", "\n", " ", ""]` → Intenta respetar párrafos, luego líneas, luego palabras



In [ ]:
# <FILL_IN> Crea el splitter con los MISMOS parámetros que definiste arriba (CHUNK_SIZE, CHUNK_OVERLAP)
# Usa las variables CHUNK_SIZE y CHUNK_OVERLAP que ya configuraste
splitter = RecursiveCharacterTextSplitter(
    chunk_size=<FILL_IN>,
    chunk_overlap=<FILL_IN>,
    separators=["\n\n", "\n", " ", ""],
)
chunks = splitter.split_documents(raw_docs)
print(f"✂️  Chunks generados: {len(chunks)}")
print(f"📈 Ratio de expansión: {len(chunks)} chunks de {len(raw_docs)} documentos")
if chunks:
    print(f"📝 Ejemplo de primer chunk (primeros 200 caracteres):\n{chunks[0].page_content[:200]}...")

## 6. Crear Embeddings y Vector Store

⏱️ **Tiempo estimado: 5-15 minutos** (según cantidad de chunks)

### Conceptos:
- **Embeddings**: Convertir texto a números (vectores)
  - El modelo de embeddings crea una representación numérica
  - Textos similares tienen vectores cercanos
- **Vector Store (Chroma)**: Base de datos de vectores
  - Almacena chunks + sus embeddings
  - Permite búsqueda semántica rápida
- **Retriever**: Interfaz para recuperar documentos similares

💡 **Pista**: Este paso es intensivo. Es normal esperar.

🎯 **Prueba de checkpoint**: El vector store debe persistirse en `./chroma_pdfs_gemini`

In [ ]:
# <FILL_IN> Crea los embeddings con el modelo configurado (usa EMBEDDING_MODEL)
embeddings = GoogleGenerativeAIEmbeddings(model=<FILL_IN>)
print("✅ Embeddings inicializados")

# <FILL_IN> Crea el vector store con Chroma
# Necesitas: documents=chunks, embedding=embeddings, persist_directory
vectorstore = Chroma.from_documents(
    documents=<FILL_IN>,
    embedding=<FILL_IN>,
    persist_directory=str(PERSIST_DIR),
)
print(f"✅ Vector store creado con {len(chunks)} chunks")

# <FILL_IN> Crea el retriever a partir del vector store
# Usa: vectorstore.as_retriever(search_kwargs={"k": TOP_K})
retriever = <FILL_IN>
print("✅ Retriever creado y listo para búsquedas")

## 7. Creamos la Tool (Retriever) para el agente

⏱️ **Tiempo estimado: 2 minutos**

### ¿Qué es una Tool?
Una herramienta que el agente puede usar durante su ejecución:
- **Nombre**: Identificador único
- **Descripción**: Qué hace (el modelo la lee para decidir si usarla)
- **Función**: El retriever que implementa la búsqueda

💡 **Pista**: Una buena descripción ayuda al modelo a saber cuándo usar esta herramienta.

In [ ]:
from langchain_core.tools.retriever import create_retriever_tool

# <FILL_IN> Crea la retriever tool
# Parámetros:
#  1. retriever (el objeto retriever que creaste)
#  2. nombre (string, ej: "search_documents")
#  3. descripción (string, ej: "Busca información en los documentos sobre...")
retriever_tool = create_retriever_tool(
    <FILL_IN>,  # retriever
    "<FILL_IN>",  # nombre de la tool (ej: "pdf_search")
    "<FILL_IN>",  # descripción (ej: "Busca información en los PDFs cargados")
)

print(f"✅ Retriever tool '{retriever_tool.name}' creada")
print(f"   Descripción: {retriever_tool.description}")

### 7.1 Test de la tool

⏱️ **Tiempo estimado: 2 minutos**

**Objetivo**: Verificar que la tool funciona correctamente
- Ejecuta una búsqueda real contra tu vector store
- Observa qué documentos se recuperan
- Valida que son relevantes

In [ ]:
# <FILL_IN> Prueba la tool con una pregunta sobre tus PDFs
# Ejemplo: "Busca en la informacion proporcionada la pregunta que hace el usuario y no inventes¿Que es un deployment tipo batch?"
resultado = retriever_tool.invoke({"query": "<FILL_IN>"})
print("\n📄 Documentos recuperados:")
print("-" * 60)
print(resultado)

## 8. Nodo: Genera query o responde directamente

⏱️ **Tiempo estimado: 5 minutos**

### Flujo del agente (agentico loop):
1. **Este nodo**: Recibe la pregunta del usuario
2. Decide: ¿Necesito buscar docs o puedo responder directamente?
3. Si usa la tool → pasa a retriever
4. Si responde → termina

### Conceptos clave:
- **bind_tools**: Conecta herramientas al modelo
- **MessagesState**: Estado que mantiene el historial de mensajes
- **ToolUse**: Cuando el modelo elige usar una herramienta

💡 **Pista**: Es el primer nodo del grafo agentico

In [ ]:
# <FILL_IN> Instancia el modelo de chat con el modelo configurado
# temperature: 0=determinista, 1=creativo. Para RAG: usa 0-0.5
response_model = ChatGoogleGenerativeAI(model=<FILL_IN>, temperature=<FILL_IN>)
print(f"✅ Modelo de chat {CHAT_MODEL} configurado")

def genera_query_o_responde(state: MessagesState):
    """Nodo 1: Decide si recuperar información o responder directamente."""
    # <FILL_IN> Conecta la tool al modelo usando bind_tools
    # Parámetros:
    #  - [lista de tools]
    #  - Luego .invoke(state["messages"])
    response = (
        response_model
        .bind_tools([<FILL_IN>]).invoke(state[<FILL_IN>])
    )
    return {"messages": [response]}

### 8.1 Test: Pregunta que no necesita búsqueda

⏱️ **Tiempo estimado: 1 minuto**

**Esperado**: El modelo responde directamente sin usar la tool
- Pregunta genérica (ej: "¿Cuál es la capital de Francia?")
- Observa que NO hay tool_use en la respuesta

In [ ]:
# <FILL_IN> Prueba con una pregunta general (NO sobre tus PDFs)
# Ej: "Explícame qué es RAG", "¿Cuál es 2+2?"
input_test = {"messages": [{"role": "user", "content": "<FILL_IN>"}]}
print("\n🧪 Test 1: Pregunta sin necesidad de búsqueda\n")
respuesta = genera_query_o_responde(input_test)
respuesta["messages"][-1].pretty_print()
print("\n✅ Observa que NO hay 'tool_use' en la respuesta")

### 8.2 Test: Pregunta que requiere búsqueda semántica

⏱️ **Tiempo estimado: 1 minuto**

**Esperado**: El modelo usa la tool para buscar documentos
- Pregunta sobre contenido de tus PDFs
- Observa que SÍ hay tool_use con el nombre de tu tool

In [ ]:
# <FILL_IN> Prueba con una pregunta sobre tus PDFs
# Ej: "¿Qué se menciona sobre...?", "Resumime el contenido sobre..."
input_test = {
    "messages": [
        {
            "role": "<FILL_IN>", # rol que comienza la accion..empieza  por "u" y termina por "ser". No puedo dar mas pistas
            "content": "<FILL_IN>",  # Pregunta sobre el contenido de tus PDFs. Buscad en los pdfs
        }
    ]
}
print("\n🧪 Test 2: Pregunta que REQUIERE búsqueda\n")
respuesta = genera_query_o_responde(input_test)
respuesta["messages"][-1].pretty_print()
print("\n✅ Observa que SÍ hay 'tool_use' (tu tool debe estar en tool_calls)")

## 9. Nodo: Evaluar relevancia de documentos

⏱️ **Tiempo estimado: 5 minutos**

### ¿Por qué evaluar relevancia?
- No siempre la búsqueda semántica recupera docs relevantes
- Control de calidad: rechazar docs no pertinentes
- Bifurcación: docs relevantes → responder, docs irrelevantes → reescribir pregunta

### GradeDocuments (Pydantic):
- Estructura de datos con score binario ("si"/"no")
- Ayuda a parsear la respuesta del modelo

💡 **Concepto**: Los modelos pueden ser instructores, pero también evaluadores

In [ ]:
# <FILL_IN> Instancia el modelo grader (puede ser el mismo que response_model)
# temperature bajo para evaluación coherente
from pydantic import BaseModel, Field

GRADE_PROMPT = (
    "Eres un evaluador que determina la relevancia de un documento recuperado respecto a una pregunta del usuario. \n "
    "Aquí tienes el documento recuperado: \n\n {context} \n\n"
    "Aquí tienes la pregunta del usuario: {question} \n"
    "Si el documento contiene palabra(s) clave o significado semántico relacionado con la pregunta del usuario, califícalo como relevante. \n"
    "Da una puntuación binaria 'si' o 'no' para indicar si el documento es relevante para la pregunta."
)

class GradeDocuments(BaseModel):
    """Califica los documentos utilizando una puntuación binaria para comprobar su relevancia"""
    binary_score: str = Field(
        description="Puntuación : 'si' si es relevante, o 'no' si no lo es"
    )

grader_model = ChatGoogleGenerativeAI(model=<FILL_IN>, temperature=<FILL_IN>)


def grade_documents(state: MessagesState) -> Literal["genera_respuesta", "rescribir_question"]:
    """Nodo 2: Evalúa si los documentos recuperados son relevantes.
    
    Retorna:
    - "genera_respuesta" si docs son relevantes
    - "rescribir_question" si no lo son
    """
    print("\n⏳ Evaluando relevancia de documentos...")
    
    # <FILL_IN> Extrae la pregunta (último mensaje de usuario)
    # Pista: es el primer elemento de state["messages"][].content
    question = <FILL_IN>
    
    # <FILL_IN> Extrae el contexto (último ToolMessage con documentos)
    # Pista: Busca en state["messages"] un mensaje que venga del retriever. 
    # Es el ultimo mensaje del elemento state["messages][].content
    context = <FILL_IN>
    
    prompt = GRADE_PROMPT.format(question=question, context=context)
    response = (
        grader_model
        .with_structured_output(GradeDocuments).invoke(
            [{"role": "user", "content": prompt}]
        )
    )
    score = response.binary_score
    print(f"📊 Score de relevancia: {score}")
    
    if score == "si":
        print("✅ Docs relevantes → generando respuesta")
        return "genera_respuesta"
    else:
        print("❌ Docs no relevantes → reescribiendo pregunta")
        return "rescribir_question"

#### 9.1 Comprobamos con una respuesta irrelevante

In [ ]:
#simulamos la respuesta de la tool mediante mensajes
from langchain_core.messages import convert_to_messages

input = {
    "messages": convert_to_messages(
        [
            {
                "role": "user",
                "content": "<FILL_IN>",#Pregunta que no tenga ningun sentido
            },
            {
                "role": "assistant",
                "content": "",
                "tool_calls": [
                    {
                        "id": "1",
                        "name": "retrieve_blog_posts",
                        "args": {"query": "¿Que es un deployment batch?"},
                    }
                ],
            },
            {"role": "tool", "content": "Son las 10 de la mañana", "tool_call_id": "1"}, #Esta respuesta que devuelve la  tool y on tiene que ver con la pregunta
        ]
    )
}


grade_documents(input)

#### 9.2 Comprobar que el documento/respuesta relevante lo clasifica como tal

In [ ]:
input = {
    "messages": convert_to_messages(
        [
            {
                "role": "user",
               
                "content": "¿Que es un deployment batch en el documento?", 
            },
            {
                "role": "assistant",
                "content": "",
                "tool_calls": [
                    {
                        "id": "1",
                        "name": "retrieve_blog_posts",
                        "args": {"query": "¿Que es un deployment batch en el documento?"},
                    }
                ],
            },
            {
                "role": "tool",
                ##El contenido de la respuesta SI es relevante en relacion a la pregunta
                "content": "El contexto proporcionado hace referencia a 'batch deployment' (despliegue por lotes) en el documento, pero no ofrece una definición explícita de qué es 'batch'. Sin embargo, se menciona que uno de los objetivos de aprendizaje es describir el despliegue en batch y sus escenarios de uso, así como identificar las ventajas y desventajas de desplegar un modelo mediante procesamiento por lotes, y discutir un flujo de trabajo típico para este tipo de despliegue en Databricks.",
                "tool_call_id": "1",
            },
        ]
    )
}
grade_documents(input)

## 10. Nodo: Rescribir la pregunta si no es clara

⏱️ **Tiempo estimado: 3 minutos**

### Flujo iterativo:
Si los docs no son relevantes:
1. Reescribir pregunta (mejorar redacción)
2. Volver a generar query
3. Buscar de nuevo
4. Evaluar de nuevo

💡 **Concepto**: Query rewriting → búsqueda mejorada → mejor contexto

⚠️ **Nota**: En un grafo real habría límite de iteraciones para evitar loops infinitos

In [ ]:
REWRITE_PROMPT = (
    "Analiza detenidamente la siguiente pregunta e intenta comprender la intención o el significado profundo que transmite.\n"
    "Pregunta original:"
    "\n ------- \n"
    "{question}"
    "\n ------- \n"
    "Ahora, reescribe la pregunta para que sea más clara, precisa y fácil de entender:"
)
def rescribir_question(state: MessagesState):
    """Nodo 3: Reescribe la pregunta del usuario para mejorarla."""
    print("\n✏️  Reescribiendo pregunta para mejorar búsqueda...")
    messages = state["messages"]
    
    # <FILL_IN> Extrae la pregunta original del usuario
    # Busca el último mensaje con role="user" en state["messages"]
    question = <FILL_IN>
    
    prompt = REWRITE_PROMPT.format(question=question)
    response = response_model.invoke([{"role": "user", "content": prompt}])
    
    print(f"📝 Pregunta reescrita: {response.content[:100]}...")
    return {"messages": [{"role": "user", "content": response.content}]}

### 10.1 Probamos la función de rescribir la pregunta

In [ ]:
input = {
    "messages": convert_to_messages(
        [
            {
                "role": "user",
                "content": "¿Que es batch?",
            },
            {
                "role": "assistant",
                "content": "",
                "tool_calls": [
                    {
                        "id": "1",
                        "name": "retrieve_blog_posts",
                        "args": {"query": "¿Que es batch?"},
                    }
                ],
            },
            {"role": "tool", "content": "Son las 10 de la mañana", "tool_call_id": "1"},#respuesta absurda --> Rescribe la pregunta
        ]
    )
}

response = rescribir_question(input)
print(response["messages"][-1]["content"])

## 11. Nodo: Generar la respuesta final

⏱️ **Tiempo estimado: 3 minutos**

### Última etapa del RAG:
1. Tienes documentos relevantes (ya evaluados)
2. Tienes la pregunta del usuario
3. Formato de prompt: pregunta + contexto
4. Modelo genera respuesta basada en docs

💡 **Prompt engineering**: El GENERATE_PROMPT es crucial
- Limita respuesta a 3 frases (concisión)
- Pide admitir ignorancia ("si no sabes, di que no sabes")

### 11.1 Construimos el nodo generate_answer

In [ ]:
def genera_respuesta(state: MessagesState):
    """Nodo 4: Genera la respuesta final basada en el contexto relevante."""
    print("\n🤖 Generando respuesta final...")
    
    # <FILL_IN> Extrae la pregunta del usuario
    question = <FILL_IN>
    
    # <FILL_IN> Extrae el contexto (documento relevante)
    # Busca el último ToolMessage con el contenido recuperado
    context = <FILL_IN>
    #Hay que formatear el prompt
    prompt = GENERATE_PROMPT.<FILL_IN>(question=question, context=context)
    #Hay que invocar al modelo con la pregunta ey el contexto
    response = response_model.<FILL_IN>([{"role": "user", "content": prompt}])
    
    print(f"\n✅ Respuesta generada: {response.content[:150]}...")
    return {"messages": [response]}

### 11.2 Comprobamos el metodo de generar respuesta

In [ ]:
input = {
    "messages": convert_to_messages(
        [
            {
                "role": "user",
                "content": "¿Qué significa el término 'batch deployment en databricks' y en qué contextos se utiliza?",
            },
            {
                "role": "assistant",
                "content": "",
                "tool_calls": [
                    {
                        "id": "1",
                        "name": "retrieve_blog_posts",
                        "args": {"query": "¿Qué significa el término 'batch deployment en databriccks' y en qué contextos se utiliza?"},
                    }
                ],
            },
            {
                "role": "tool",
                "content": "El contexto proporcionado hace referencia a 'batch deployment' (despliegue por lotes) en el documento, pero no ofrece una definición explícita de qué es 'batch'. Sin embargo, se menciona que uno de los objetivos de aprendizaje es describir el despliegue en batch y sus escenarios de uso, así como identificar las ventajas y desventajas de desplegar un modelo mediante procesamiento por lotes, y discutir un flujo de trabajo típico para este tipo de despliegue en Databricks",
                "tool_call_id": "1",
            },
        ]
    )
}

response = genera_respuesta(input)
response["messages"][-1].pretty_print()

## 12. Construir el grafo (workflow agentico)

⏱️ **Tiempo estimado: 5-7 minutos**

### Estructura del grafo:
```
START → genera_query_o_responde
         ↓
    ¿Usa tool?
    /        \
  si         no → END
  ↓
retrieve (ejecuta tool)
  ↓
grade_documents (evalúa relevancia)
  /          \
si           no
↓            ↓
genera_respuesta  rescribir_question
↓              ↓
END ←─────────────────────────────┘
```

### Componentes:
- **Nodos**: Funciones que ejecutan lógica
- **Aristas**: Conexiones entre nodos
- **Aristas condicionales**: Decisiones basadas en salida

💡 **Pista**: Los nombres de nodos deben coincidir en todo el código

### 12.1 Importamos los elementos necesarios para construir el grafo

In [ ]:
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition

#### 12.2. Ensamblamos el workflow

#### 12.2.1 Añadimos los nodos.
![Nodos del Grafo](./images/grafo_nodos.jpg)

In [ ]:
print("\n🏗️  Construyendo el grafo agentico...\n")

# <FILL_IN> Crea el StateGraph con el estado que usamos (MessagesState)
workflow = StateGraph(<FILL_IN>)

# <FILL_IN> Añade los nodos (nombre, función)
workflow.add_node(<FILL_IN>, genera_query_o_responde)  # Nodo 1
workflow.add_node("retrieve", ToolNode([<FILL_IN>]))  # Ejecutor de tools
workflow.add_node(<FILL_IN>, rescribir_question)  # Nodo 3
workflow.add_node(<FILL_IN>, genera_respuesta)  # Nodo 4


#### 12.2.2 Añadimos las aristas.
![Aristas del Grafo](./images/grafo_aristas.jpg)

In [ ]:
# <FILL_IN> Arista inicial: START → genera_query_o_responde
workflow.add_edge(<FILL_IN>, "<FILL_IN>")  # START → nombreDelPrimerNodo

# <FILL_IN> Arista condicional: Si usa herramienta → retrieve, sino → END
workflow.add_conditional_edges(
    "genera_query_o_responde",
    tools_condition,  # Función que decide basada en tool_use
    {
        "tools": "retrieve",
        END: END,
    },
)

# <FILL_IN> Arista condicional desde retrieve: grade_documents decide el siguiente
workflow.add_conditional_edges(
    "retrieve",
    <FILL_IN>,  # Función: grade_documents
)

# <FILL_IN> Aristas simples
workflow.add_edge("genera_respuesta", <FILL_IN>)  # genera_respuesta → END
workflow.add_edge("rescribir_question", "genera_query_o_responde")  # loop

print("✅ Grafo construido correctamente")

## 13. Compilar el grafo

⏱️ **Tiempo estimado: 1 minuto**

### ¿Qué significa compilar?
Convertir la estructura de nodos/aristas en un ejecutable
- Valida conexiones
- Prepara para ejecución

💡 **Si hay error aquí**: Revisa nombres de nodos, aristas, y tipos de estado

In [ ]:
# <FILL_IN> Compila el grafo con workflow.compile()
graph = <FILL_IN>.compile()
print("✅ Grafo compilado y listo para ejecutar")

## 14. Visualizar el grafo

⏱️ **Tiempo estimado: 1 minuto**

### Diagrama visual:
- Ayuda a entender el flujo
- Facilita debugging
- Muestra nodos, aristas, condicionales

💡 **Tip**: Si no se visualiza, puede ser problema de dependencias (graphviz)

In [ ]:
from IPython.display import Image, display

print("\n📊 Visualizando el grafo agentico...\n")

# <FILL_IN> Visualiza el grafo con graph.get_graph().draw_mermaid_png()
display(Image(graph.get_graph().draw_mermaid_png()))

## 15. Ejecutar el grafo - Modo DEBUG

⏱️ **Tiempo estimado: 5-10 minutos** (depende de pregunta)

### Este modo:
- Muestra cada nodo por el que pasa
- Imprime todos los mensajes intermedios
- Útil para debugging y entender el flujo

🎯 **Objetivo**: Ver exactamente cómo se ejecuta el agente

In [ ]:
from pprint import pprint

print("\n🚀 Ejecutando el grafo en modo DEBUG...\n")

# <FILL_IN> Define tu pregunta (sobre tus PDFs)
pregunta = "<FILL_IN>"
print(f"❓ Pregunta: {pregunta}\n")
print("=" * 60)

# Ejecuta el grafo paso a paso
for chunk in graph.stream({"messages": [{"role": "user", "content": pregunta}]}):
    for node, update in chunk.items():
        print(f"\n📘 Nodo ejecutado: {node}")
        print("-" * 60)
        messages = update.get("messages", [])
        if messages:
            last_msg = messages[-1]
            try:
                if hasattr(last_msg, "content"):
                    print("📝 Contenido:")
                    print(last_msg.content[:300] if len(last_msg.content) > 300 else last_msg.content)
                elif hasattr(last_msg, "tool_calls"):
                    print(f"🔧 Tool calls: {[tc.name for tc in last_msg.tool_calls]}")
                else:
                    pprint(last_msg)
            except Exception as e:
                print(f"❌ Error: {str(e)}")
        print("-" * 60)

print("\n" + "=" * 60)
print("✅ Ejecución completada")

## 16. Ejecutar el grafo - Modo PRODUCCIÓN

⏱️ **Tiempo estimado: 5-10 minutos**

### Este modo:
- Solo muestra la respuesta final
- Modo limpio (sin debug)
- Usa formateo Markdown para mejor visualización

🎯 **Objetivo**: Resultado final en forma de usuario

In [ ]:
from IPython.display import display, Markdown

print("\n🚀 Ejecutando el grafo en modo PRODUCCIÓN...\n")

# <FILL_IN> Define tu pregunta (sobre tus PDFs)
pregunta = "<FILL_IN>"
print(f"❓ Pregunta: {pregunta}\n")
print("=" * 60)

# Ejecuta el grafo y muestra solo la respuesta final
resultado = graph.invoke({"messages": [{"role": "user", "content": pregunta}]})

print("\n📝 RESPUESTA FINAL:\n")
display(Markdown(resultado["messages"][-1].content))

print("\n" + "=" * 60)
print("✅ Consulta completada")

## 17. Resumen y Conceptos Clave

⏱️ **Tiempo estimado: 5-7 minutos** (discusión/preguntas)

### Flujo RAG Agentico completo:
1. **Cargar** (secs 1-4): Documentos → PDFs
2. **Chunking** (sec 5): Documentos → Chunks (piezas manejables)
3. **Embeddings** (sec 6): Chunks → Vectores (números)
4. **Nodo 1** (sec 8): ¿Buscar o responder?
   - Si buscar → Nodo 2 (retrieve)
   - Si responder → END
5. **Nodo 2** (sec 6-7): Buscar documentos similares
6. **Nodo Evaluador** (sec 9): ¿Son relevantes?
   - Si relevantes → Nodo 4 (responder)
   - Si no → Nodo 3 (reescribir pregunta)
7. **Nodo 3** (sec 10): Mejorar pregunta → Volver a Nodo 1
8. **Nodo 4** (sec 11): Generar respuesta final → END

### Conceptos clave aprendidos:
- **Embeddings**: Representación numérica del texto
- **Vector Store**: Base de datos de búsqueda semántica
- **Agentic Loop**: Ciclo de decisión automático
- **Retrieval Augmented Generation (RAG)**: Mejorar respuestas con contexto
- **Graph State**: Máquina de estados con LangGraph
- **Tool Use**: Modelos pueden llamar herramientas automáticamente

### Tips para mejorar:
1. **Tuning de chunks**: Experimenta con CHUNK_SIZE y CHUNK_OVERLAP
2. **Mejor prompt engineering**: Mejorar GRADE_PROMPT y GENERATE_PROMPT
3. **Multiple retrievers**: Combinar BM25 + Semántico
4. **Caching**: Guardar embeddings calculados
5. **Evaluación**: Metrics como precision@k, NDCG

### Siguientes pasos:
- 🔧 Personaliza los prompts para tu caso de uso
- 📊 Mide la calidad de respuestas (F1-score, BLEU, etc)
- 🚀 Despliega en producción con FastAPI/Streamlit
- 📈 Monitorea performance en tiempo real

### Imágenes originales de referencia

#### Grafo - Nodos

![Nodos del Grafo](./images/grafo_nodos.jpg)

#### Grafo - Aristas y Flujo

![Aristas del Grafo](./images/grafo_aristas.jpg)

## 18. Troubleshooting y Preguntas Frecuentes

### ❌ Errores comunes y soluciones:

#### 1. "GOOGLE_API_KEY not found"
- **Solución**: 
  - Crea archivo `.env` en la carpeta con: `GOOGLE_API_KEY=tu_clave_aqui`
  - O exporta en terminal: `$env:GOOGLE_API_KEY='tu_clave'`

#### 2. "No documents loaded"
- **Solución**:
  - Verifica que `./docs` existe y tiene PDFs
  - Prueba: `ls ./docs` o `Get-ChildItem ./docs`
  - PDFs deben ser válidos (no corruptos)

#### 3. "Retriever returns empty results"
- **Solución**:
  - Chunks demasiado grandes: baja CHUNK_SIZE
  - Consulta muy diferente: usa similar query a tu contenido
  - Vector store vacío: regenera con `Chroma.from_documents(...)`

#### 4. "Tool not used by model"
- **Solución**:
  - Descripción de tool no clara: mejora `description`
  - Pregunta no requiere búsqueda: haz pregunta sobre PDFs
  - Modelo no reconoce tool: verifica `bind_tools([tool_name])`

#### 5. "Model rejects structured output"
- **Solución**:
  - Usa modelo más reciente: "gemini-2.5-pro"
  - Simplifica GradeDocuments (menos campos)
  - Usa `.with_structured_output()` correctamente

### 💡 Tips de debugging:

```python
# Ver estructura de state
print("State keys:", list(state.keys()))
print("Messages:", [m.get("role") for m in state["messages"]])

# Debug embeddings
test_embedding = embeddings.embed_query("test")
print(f"Embedding dimension: {len(test_embedding)}")

# Ver chunks generados
for i, chunk in enumerate(chunks[:3]):
    print(f"Chunk {i}: {len(chunk.page_content)} chars")
```

### 📚 Recursos útiles:

- [LangChain Docs](https://python.langchain.com/)
- [LangGraph Docs](https://langchain-ai.github.io/langgraph/)
- [Google Gemini API](https://ai.google.dev/)
- [Chroma Docs](https://docs.trychroma.com/)